# Extract the Travel Time for the Locations of Different Properties

## Import Packages

In [115]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict


# Database Connection
from sqlalchemy import create_engine
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

## Other Setup

In [116]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
# this allows one to reload the custom package without having to install it again
%autoreload 1

sys.path.insert(0,'../src/')

# Import the custom package and sub-packages
%aimport rental_utils
%aimport rental_utils.functions
%aimport rental_utils.sql_queries

In [118]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

### Import Custom Packages

In [119]:
from rental_utils import sql_queries as sqlq
from rental_utils import functions as rent

### Finding the correctd file directory for the credentials json with the api key

In [120]:
current_dir = os.path.dirname(os.path.abspath("NB03_Extract_Travel_Time_Data.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "credentials.json")

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [121]:
pprint(credentials)

{'api_key': '716417c04acf3944937af85d78775ea0', 'app_id': '27beab81'}


## Augment Property Data with Commute Times

### Load in the Property Data

#### Connect to the Database

In [122]:
engine = sqlq.get_sql_engine("../data/properties.db")

#### Load in the Data from the Corresponding Table in the Database

In [123]:
with engine.connect() as connection:
    rightmove_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
rightmove_data.head()

,id,price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,161782598,1150.000000,None,None,2,2,10,"Martello Street, Hackney, London",51.542839,-0.057569,Flat,price_reduced,2025-06-30T16:34:40Z,2300,monthly,0,0,rent,0,65 sq. m.,/properties/161782598#/?channel=RES_LET,2025-05-10T13:15:57Z,Reduced today,2 bedroom flat
1,161786591,766.666667,None,None,3,2,12,"Penrhyn Avenue, London, E17 5BJ",51.598030,-0.020891,End of Terrace,price_reduced,2025-06-30T16:35:49Z,2300,monthly,1,0,rent,0,90 sq. m.,/properties/161786591#/?channel=RES_LET,2025-05-10T16:30:55Z,Reduced today,3 bedroom end of terrace house
2,162252746,346.000000,None,None,2,1,4,"Long Street, London, Shoredtich, E2",51.529050,-0.076430,Apartment,price_reduced,2025-06-30T16:35:02Z,692,weekly,0,0,rent,0,,/properties/162252746#/?channel=RES_LET,2025-05-21T15:30:03Z,Reduced today,2 bedroom apartment
3,162753398,800.000000,None,None,3,1,12,"Folkestone Road, London",51.585121,-0.016140,Flat,price_reduced,2025-06-30T16:36:12Z,2400,monthly,1,0,rent,0,80 sq. m.,/properties/162753398#/?channel=RES_LET,2025-06-02T20:53:51Z,Reduced today,3 bedroom flat
4,163298408,1025.000000,None,None,2,3,7,"Hoptree Close, N12",51.617380,-0.185520,Flat,price_reduced,2025-06-30T16:35:11Z,2050,monthly,0,0,rent,0,,/properties/163298408#/?channel=RES_LET,2025-06-13T15:34:39Z,Reduced today,2 bedroom flat


### Extract the Commute Times for A Chosen Location

#### Set Up the Headers

In [124]:
headers = {
    "Content-Type": "application/json",
    "X-Application-Id": credentials["app_id"],
    "X-Api-Key": credentials["api_key"]
}

#### Define a Function that Takes the Property IDs, Latitudes and Longitudes, and Creates a Custome Payload for the TravelTime API

In [125]:
def create_payload(df: pd.DataFrame, search_id: str="1", transportation_type: str = "public_transport") -> dict:
    """
    Creates a payload dictionary for the TravelTime API using property locations from a DataFrame.
    The payload includes an origin (Bank Station) and destination locations (properties), 
    and sets up the search parameters for a one-to-many public transport commute time query.
    """
    # Define origin (Bank Station - a key commuting hub)
    origin = {
        "id": "Origin",
        "coords": {"lat": 51.513, "lng": -0.088}
    }
    # Ensure the 'id' column is of string type for API compatibility
    df["id"] = df["id"].astype(str)

    # Select and rename latitude/longitude columns for API format
    locations = df[["id", "latitude", "longitude"]].rename(
        columns={"latitude": "lat", "longitude": "lng"}
    )

    # Convert DataFrame rows to a list of dicts for each destination
    destinations = locations.to_dict(orient="records")
    destination_locations = [
        {
            "id": d["id"],
            "coords": {"lat": d["lat"], "lng": d["lng"]}
        } for d in destinations
    ]

    # Build the final payload structure for the API request
    payload = {
        "arrival_searches": {
            "one_to_many": [
                {
                    "id": search_id,  # Unique search identifier
                    "departure_location_id": "Origin",  # Start from Bank Station
                    "arrival_location_ids": df["id"].tolist(),  # List of property IDs as destinations
                    "transportation": {"type": transportation_type},  # Mode of transport
                    "travel_time": 10800,  # Max travel time in seconds (3 hours)
                    "arrival_time_period": "weekday_morning",  # Commute time window
                    "properties": ["travel_time", "distance"]  # Data to return
                }
            ]
        },
        "locations": [origin] + destination_locations  # All locations (origin + destinations)
    }

    return payload

In [126]:
payload_1 = create_payload(rightmove_data)
pprint(payload_1)

{'arrival_searches': {'one_to_many': [{'arrival_location_ids': ['161782598',
                                                                '161786591',
                                                                '162252746',
                                                                '162753398',
                                                                '163298408',
                                                                '163546691',
                                                                '163587311',
                                                                '163818515',
                                                                '163948796',
                                                                '163976093',
                                                                '163976414',
                                                                '163976642',
                                                                '163976705',

In [127]:
# Make the request
response = requests.post(
    "https://api.traveltimeapp.com/v4/time-filter/fast",
    headers= headers,
    data=json.dumps(payload_1)
)

# Check the response
print(response.status_code)
pprint(response.json())  # or response.text if not JSON

200
{'results': [{'locations': [{'id': '163976786',
                             'properties': {'distance': 0,
                                            'travel_time': 1739}},
                            {'id': '163976984',
                             'properties': {'distance': 0,
                                            'travel_time': 1649}},
                            {'id': '163298408',
                             'properties': {'distance': 0,
                                            'travel_time': 2578}},
                            {'id': '163976768',
                             'properties': {'distance': 0,
                                            'travel_time': 1533}},
                            {'id': '163976777',
                             'properties': {'distance': 0,
                                            'travel_time': 2066}},
                            {'id': '163976093',
                             'properties': {'distance': 0, 'travel_time': 894}

In [128]:
response.status_code == 200

True

### Convert the Response into a New Dataframe with info on Travel Times and Distances for each Property

In [129]:
results = response.json()["results"][0]["locations"]

# Convert to DataFrame
df_results = pd.DataFrame([
    {
        "id": loc["id"],
        "distance": loc["properties"]["distance"],
        "travel_time": loc["properties"]["travel_time"]
    }
    for loc in results
])

# inspect the results
df_results.head(3)

,id,distance,travel_time
0,163976786,0,1739
1,163976984,0,1649
2,163298408,0,2578


### Merge Back With all the Other Original Data about Properties

In [130]:
# Conduct the merge
properties_data = df_results.merge(rightmove_data.drop(columns=["travel_time","distance"]), on="id", how="left")
# Check the data
properties_data.head(3)

,id,distance,travel_time,price_per_bed,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,163976786,0,1739,230.666667,3,1,11,"Warwick Crescent, Little Venice, W9",51.52068,-0.18288,Apartment,new,2025-06-30T16:36:02Z,692,weekly,0,0,rent,0,,/properties/163976786#/?channel=RES_LET,2025-06-30T16:30:06Z,Added today,3 bedroom apartment
1,163976984,0,1649,1050.000000,2,1,8,Conington Road London SE13,51.46767,-0.01394,Ground Flat,new,2025-06-30T16:38:17Z,2100,monthly,0,0,rent,0,66 sq. m.,/properties/163976984#/?channel=RES_LET,2025-06-30T16:32:12Z,Added today,2 bedroom ground floor flat
2,163298408,0,2578,1025.000000,2,3,7,"Hoptree Close, N12",51.61738,-0.18552,Flat,price_reduced,2025-06-30T16:35:11Z,2050,monthly,0,0,rent,0,,/properties/163298408#/?channel=RES_LET,2025-06-13T15:34:39Z,Reduced today,2 bedroom flat


#### Save the Data the Database Into the Old Table

In [131]:
# Save the filled-in dataframe into the pre-existing table, replacing the incomplete with the complete data
sqlq.make_table(properties_data, "properties_data", engine, if_exists='replace')


### Check if it can be Extracted OK

In [132]:
with engine.connect() as connection:
    properties_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
properties_data.head()

,id,distance,travel_time,price_per_bed,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,163976786,0,1739,230.666667,3,1,11,"Warwick Crescent, Little Venice, W9",51.52068,-0.182880,Apartment,new,2025-06-30T16:36:02Z,692,weekly,0,0,rent,0,,/properties/163976786#/?channel=RES_LET,2025-06-30T16:30:06Z,Added today,3 bedroom apartment
1,163976984,0,1649,1050.000000,2,1,8,Conington Road London SE13,51.46767,-0.013940,Ground Flat,new,2025-06-30T16:38:17Z,2100,monthly,0,0,rent,0,66 sq. m.,/properties/163976984#/?channel=RES_LET,2025-06-30T16:32:12Z,Added today,2 bedroom ground floor flat
2,163298408,0,2578,1025.000000,2,3,7,"Hoptree Close, N12",51.61738,-0.185520,Flat,price_reduced,2025-06-30T16:35:11Z,2050,monthly,0,0,rent,0,,/properties/163298408#/?channel=RES_LET,2025-06-13T15:34:39Z,Reduced today,2 bedroom flat
3,163976768,0,1533,1050.000000,1,1,16,"Alexandra Street, E16",51.51847,0.018110,House of Multiple Occupation,new,2025-06-30T16:35:06Z,1050,monthly,0,0,rent,0,,/properties/163976768#/?channel=RES_LET,2025-06-30T16:30:00Z,Added today,1 bedroom house of multiple occupation
4,163976777,0,2066,1100.000000,2,1,9,"Lisgar Terrace, West Kensington, West Kensington",51.49410,-0.208574,Flat,new,2025-06-30T16:35:06Z,2200,monthly,0,0,rent,0,,/properties/163976777#/?channel=RES_LET,2025-06-30T16:30:02Z,Added today,2 bedroom flat
